In [1]:
from pathlib import Path
from langchain_community.document_loaders import TextLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import torch

In [2]:
torch.cuda.is_available()

True

In [4]:
splitting_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={
        'device': 'cuda',
    },
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
text_splitter = SemanticChunker(
    embeddings=splitting_embeddings)

In [6]:
docs_dir = Path("/home/bartek/Kod/PD/praca_dyplomowa/dane/texts/cleaned_texts")
documents = []

In [12]:
for txt_file in list(docs_dir.glob("**/LLM_clean_text*.txt"))[:10]:
    print(txt_file.parts[-2])

Taiwan_Government_Targeted_by_Multiple_Cyberattacks_in_April_2020_Part_2:_Owlproxy_Malware
Bzz.._Bzz.
Prying_Libra
Beware_Android_trojan_posing_as_Clubhouse_app
Reflections_of_the_Israel-Palestine_Conflict_on_the_Cyber_World
RAT_Gone_Rogue:_Meet_ARS_VBS_Loader
TeaBot_Banking_Trojan_Posted_as_QR_Code_app_in_Google_Play_Store_Targeting_US_Users
Joker
Qakbot_Botnet_Disruption
Mustang_Panda’s_PlugX_new_variant_targetting_Taiwanese_government_and_diplomats


In [7]:
for txt_file in docs_dir.glob("**/LLM_clean_text*.txt"):
    loader = TextLoader(str(txt_file), encoding="utf-8")
    loaded_docs = loader.load()
    
    for doc in loaded_docs:
        doc.metadata["title"] = txt_file.parts[-2]
        
    documents.extend(loaded_docs)

print(f"Loaded {len(documents)} documents")

Loaded 8912 documents


In [8]:
split_docs = text_splitter.split_documents(documents)

In [9]:
del splitting_embeddings
torch.cuda.empty_cache()

In [13]:
embeddings = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-modernbert-base",
    model_kwargs={
        'device': 'cuda',
        'trust_remote_code': True
    },
    encode_kwargs={'normalize_embeddings': True}
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [14]:
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    collection_name="malpedia_vectors",
    persist_directory="./chroma_db"
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 8.00 GiB. GPU 0 has a total capacity of 15.57 GiB of which 7.27 GiB is free. Including non-PyTorch memory, this process has 7.12 GiB memory in use. Of the allocated memory 6.95 GiB is allocated by PyTorch, and 21.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)